<a href="https://colab.research.google.com/github/m-haghighi/Topic-Modeling/blob/main/SKlearn_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Import Libraries

In [ ]:
# for text preprocessing
import re
import spacy
import nltk
import pandas as pd
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# import numpy for matrix operation
import numpy as np

# import LDA from sklearn
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
#nlp = spacy.load('en_core_web_lg')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop =  set(open('/content/5303 stop-words_updated_V1.txt').read().split())
#print(Istopwords)

#stop = set(stopwords.words('english'))
stop1={'ii','The','s',',','the','to','t','v','iii','a','is','out','an','and','of',"'","",'“','”','’','[',']',"al",'\n','.',','}
stop.update(stop1)
print(stop)

{'', 'whether', 'agreement', 'whole', 'extended', 'qv', 'buy', 'idea', 'something', 'technique', 'captured', 'fully', 'til', 'also', 'revolutionary', 'obtained', 'result', 'di', 'temporal', 'rez', 'variable', 'defined', 'robustly', 'enviromental', 'parameter', '20', 'started', 'compared', 'think', 'anyways', 'page', 'pivotal', 'particular', 'applied', '2013', 'monsuwe', 'todaro', 'inclusive', 'class', 'innovativeness', 'aside', 'index', 'innovate', 'p', 'call', 'far', 'indicator', 'small', 'constantly', 'meuter', 'integrated', 'sometimes', 'doe', 'even', 'improved', '1990', '249', 'changing', 'task', 'phd', 'involving', 'testing', 'ought', 'focal', 'caliber', "we've", 'too', 'nondisclosure', 'step', 'continuous', 'focus', 'context', 'lately', 'narula', 'whomever', 'itself', 'future', 'generated', 'application', 'became', 'seems', "it'll", 'edu', 'nyawa', 'kim', 'negative', 'werent', 'broader', 'nil', 'several', 'extant', 'm', 'whereafter', 'rogers', 'volatile', 'information', '’', 'era

In [ ]:
from numpy.ma.core import shape
from collections import Counter
df=pd.DataFrame(pd.read_csv('/content/A2_Corpus_G11.csv'))

corpus=(df.content.values)
corpus_T=[]

### 2. Text Preprocessing

Steps to preprocess text data:

1. Convert the text into lowercase
2. Split text into words
3. Remove the stop loss words
3. Remove the Punctuation, any symbols and special characters
4. Normalize the word

In [ ]:
import spacy
from collections import Counter
from string import punctuation
# lemmatization
lemma = WordNetLemmatizer() 
nlp = spacy.load("en")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN','VERB'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation or token.text in stop):
            continue
        if(token.pos_ in pos_tag):
            result.append(lemma.lemmatize(token.text))
    return result
clean_corpus = [get_hotwords(doc) for doc in corpus ]
print(clean_corpus)

[['solution', 'problem', 'meaning', 'keep', 'delighted', 'discovery', 'direction'], ['meaningful', 'communicate', 'meaning'], ['meaning', 'divided', 'symbolic', 'meaning', 'researcher', 'symbolic', 'researcher', 'categorization', 'definition', 'consensus', 'product', 'pure', 'functionality', 'utility', 'intangible', 'value', 'intangible', 'value', 'socio', 'symbolism', 'experience'], ['order', 'differentiate', 'type', 'norman', 'verganti', 'define', 'incremental', 'improvement', 'solution', 'radical', 'change'], ['enablers', 'barrier', 'study', 'swedish', 'furniture', 'company', 'insight', 'enablers', 'barrier', 'issue', 'development', 'link', 'demonstrate', 'trade', 'offs', 'furnitureco', 'encounter', 'development', 'color', 'material', 'matrix', 'company', 'manage', 'furniture', 'development', 'form', 'establishing', 'collaboration', 'designer'], ['enablers', 'barrier', 'connection', 'interaction', 'relevance', 'meaning'], ['limitation', 'material', 'choice', 'flexibility', 'supplier

In [ ]:
clean_corpus

[['solution',
  'problem',
  'meaning',
  'keep',
  'delighted',
  'discovery',
  'direction'],
 ['meaningful', 'communicate', 'meaning'],
 ['meaning',
  'divided',
  'symbolic',
  'meaning',
  'researcher',
  'symbolic',
  'researcher',
  'categorization',
  'definition',
  'consensus',
  'product',
  'pure',
  'functionality',
  'utility',
  'intangible',
  'value',
  'intangible',
  'value',
  'socio',
  'symbolism',
  'experience'],
 ['order',
  'differentiate',
  'type',
  'norman',
  'verganti',
  'define',
  'incremental',
  'improvement',
  'solution',
  'radical',
  'change'],
 ['enablers',
  'barrier',
  'study',
  'swedish',
  'furniture',
  'company',
  'insight',
  'enablers',
  'barrier',
  'issue',
  'development',
  'link',
  'demonstrate',
  'trade',
  'offs',
  'furnitureco',
  'encounter',
  'development',
  'color',
  'material',
  'matrix',
  'company',
  'manage',
  'furniture',
  'development',
  'form',
  'establishing',
  'collaboration',
  'designer'],
 ['enab

### 3. Convert Text into Numerical Representation

Converting the clean preprocessed corpus to array

In [ ]:
# Converting text into numerical representation
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)

# Converting text into numerical representation
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

In [ ]:
# Array from TF-IDF Vectorizer 
tf_idf_arr = tf_idf_vectorizer.fit_transform(clean_corpus)

# Array from Count Vectorizer 
cv_arr = cv_vectorizer.fit_transform(clean_corpus)

In [ ]:
# this is our converted text to numerical representation from the Tf-IDF vectorizer

tf_idf_arr

<470x2581 sparse matrix of type '<class 'numpy.float64'>'
	with 6978 stored elements in Compressed Sparse Row format>

In [ ]:
# this is our converted text to numerical representation from the Count vectorizer
cv_arr

<470x2581 sparse matrix of type '<class 'numpy.int64'>'
	with 6978 stored elements in Compressed Sparse Row format>

The corpus has 52 columns and 5 rows corresponding to our document and 58 represents the unique Vocabulary present in our corpus.

In [ ]:
# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tf_idf_vectorizer.get_feature_names()

# get the vocb list
vocab_tf_idf

['2000).(adinyira',
 '2010s',
 '2016,p.1',
 '21st',
 'a.k.a',
 'abandonment',
 'ability',
 'abroad',
 'absence',
 'absent',
 'absorbed',
 'abstraction',
 'academic',
 'accelerate',
 'accelerated',
 'accelerating',
 'acceleration',
 'acceptable',
 'acceptance',
 'accepted',
 'accepts',
 'access',
 'accessibility',
 'accessible',
 'accommodate',
 'accomplish',
 'accomplishes',
 'accomplishing',
 'accountancy',
 'accrued',
 'accumulated',
 'accuracy',
 'achieve',
 'achieved',
 'achieves',
 'acknowledgement',
 'acquaintance',
 'acquire',
 'acquired',
 'acquirer',
 'acquiring',
 'acquisition',
 'acqusitions',
 'acting',
 'action',
 'actionable',
 'activation',
 'active',
 'activi',
 'activitis',
 'activity',
 'actor',
 'ad\x02vantage',
 'adaptability',
 'adaptation',
 'adaption',
 'adapts',
 'add',
 'addiction',
 'additive',
 'address',
 'addressed',
 'addressing',
 'adidas',
 'adjacent',
 'adjustable',
 'adjusting',
 'adjuvant',
 'adner',
 'adopter',
 'adopting',
 'adopts',
 'advan\x02tage

In [ ]:
# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

# get the vocb list
vocab_cv

['2000).(adinyira',
 '2010s',
 '2016,p.1',
 '21st',
 'a.k.a',
 'abandonment',
 'ability',
 'abroad',
 'absence',
 'absent',
 'absorbed',
 'abstraction',
 'academic',
 'accelerate',
 'accelerated',
 'accelerating',
 'acceleration',
 'acceptable',
 'acceptance',
 'accepted',
 'accepts',
 'access',
 'accessibility',
 'accessible',
 'accommodate',
 'accomplish',
 'accomplishes',
 'accomplishing',
 'accountancy',
 'accrued',
 'accumulated',
 'accuracy',
 'achieve',
 'achieved',
 'achieves',
 'acknowledgement',
 'acquaintance',
 'acquire',
 'acquired',
 'acquirer',
 'acquiring',
 'acquisition',
 'acqusitions',
 'acting',
 'action',
 'actionable',
 'activation',
 'active',
 'activi',
 'activitis',
 'activity',
 'actor',
 'ad\x02vantage',
 'adaptability',
 'adaptation',
 'adaption',
 'adapts',
 'add',
 'addiction',
 'additive',
 'address',
 'addressed',
 'addressing',
 'adidas',
 'adjacent',
 'adjustable',
 'adjusting',
 'adjuvant',
 'adner',
 'adopter',
 'adopting',
 'adopts',
 'advan\x02tage

In [ ]:
display(len(vocab_tf_idf))
display(len(vocab_cv))

2581

2581

### 4. Implementation of LDA

To implement LDA, pass the corpus: document-term matrix to the model. We had above obtained the unique words of vocabulary using both TF-IDF and Count Vectorizer. 

In [ ]:
 # Implementation of LDA:
    
# Create object for the LDA class 
# Inside this class LDA: define the components:
lda_model = LatentDirichletAllocation(n_components =35, max_iter =5, random_state = 2)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(tf_idf_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_

### 4a. Retrieve the Topics


In [ ]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words =10

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['offering' 'customer' 'cpbmi' 'change' 'capability' 'plan' 'elemental'
 'comparative' 'behaviour']
Topic 2 ['cross' 'border' 'customer' 'company' 'cvp' 'firm' 'help' 'schema'
 'position']
Topic 3 ['proposition' 'company' 'committed' 'border' 'scale' 'operate' 'business'
 'take' 'partnering']
Topic 4 ['capability' 'firm' 'resource' 'moderator' 'sme' 'competitor'
 'performance' 'partner' 'digital']
Topic 5 ['proposition' 'form' 'entrepreneur' 'apps' 'multiplicity' 'necessitates'
 'newcomer' 'establishment' 'business']
Topic 6 ['extract' 'firm' 'benefit' 'patient' 'proposition' 'advertising'
 'rational' 'occupy' 'terminal']
Topic 7 ['customer' 'partner' 'offering' 'dimension' 'market' 'activity' 'capture'
 'engagement' 'company']
Topic 8 ['threat' 'sbmi' 'meaningful' 'healthcare' 'model' 'communicate' 'meaning'
 'categorized' 'iot']
Topic 9 ['opportunity' 'customer' 'pressure' 'reflect' 'perceived' 'model' 'order'
 'altered' 'grow']
Topic 10 ['learning' 'digital' 'mode' 'customer

Above is the words per topic. 

### 4b. Annotating the topics the documents

In [ ]:
# To view what topics are assigned to the douments:

doc_topic = lda_model.transform(tf_idf_arr)  

# iterating over ever value till the end value
for n in range(doc_topic.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    
    # document is n+1  
    print ("Document", n+1, " -- Topic:" ,topic_doc)

Document 1  -- Topic: 13
Document 2  -- Topic: 1
Document 3  -- Topic: 15
Document 4  -- Topic: 1
Document 5  -- Topic: 28
Document 6  -- Topic: 11
Document 7  -- Topic: 34
Document 8  -- Topic: 25
Document 9  -- Topic: 19
Document 10  -- Topic: 11
Document 11  -- Topic: 13
Document 12  -- Topic: 19
Document 13  -- Topic: 30
Document 14  -- Topic: 30
Document 15  -- Topic: 25
Document 16  -- Topic: 27
Document 17  -- Topic: 21
Document 18  -- Topic: 28
Document 19  -- Topic: 31
Document 20  -- Topic: 0
Document 21  -- Topic: 16
Document 22  -- Topic: 26
Document 23  -- Topic: 1
Document 24  -- Topic: 7
Document 25  -- Topic: 16
Document 26  -- Topic: 20
Document 27  -- Topic: 4
Document 28  -- Topic: 5
Document 29  -- Topic: 3
Document 30  -- Topic: 7
Document 31  -- Topic: 13
Document 32  -- Topic: 28
Document 33  -- Topic: 1
Document 34  -- Topic: 6
Document 35  -- Topic: 30
Document 36  -- Topic: 16
Document 37  -- Topic: 25
Document 38  -- Topic: 19
Document 39  -- Topic: 29
Docume

This is the final output which gives us the topic along with the documents.